In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import display, clear_output, Audio
from torch.utils.data import DataLoader
from datasets import load_dataset
from dance.audio import RateDistortionAutoEncoder
import fastprogress
from fastprogress.fastprogress import master_bar, progress_bar
import matplotlib.pyplot as plt
from ipywidgets import Output

/home/dgj335/.local/lib/python3.10/site-packages/beartype/_util/error/utilerrwarn.py:67: BeartypeModuleUnimportableWarning: Ignoring module "onnx" importation exception:
    ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/home/dgj335/.local/lib/python3.10/site-packages/google/protobuf/internal/__init__.py)
  warn(message, cls)
/home/dgj335/.local/lib/python3.10/site-packages/beartype/_util/error/utilerrwarn.py:67: BeartypeModuleUnimportableWarning: Ignoring module "onnx" importation exception:
    ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/home/dgj335/.local/lib/python3.10/site-packages/google/protobuf/internal/__init__.py)
  warn(message, cls)
/home/dgj335/.local/lib/python3.10/site-packages/beartype/_util/error/utilerrwarn.py:67: BeartypeModuleUnimportableWarning: Ignoring module "onnx" importation exception:
    ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/home/dgj335/.local/lib/python3.10/si

In [ ]:
train_dataset = load_dataset("danjacobellis/aria_ea_audio_preprocessed",split="train").with_format("torch")
valid_dataset = load_dataset("danjacobellis/aria_ea_audio_preprocessed",split="validation").with_format("torch")

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
test_sample = valid_dataset[10]['audio'].unsqueeze(0)
test_input = test_sample.permute(0,2,1).to("cuda")
Audio(test_sample[0,:,0],rate=48000)

In [ ]:
plt.plot(test_sample[0,0:1000,[0,6]])

In [ ]:
model = RateDistortionAutoEncoder().to("cuda")
parameters = set(p for n, p in model.named_parameters() if not n.endswith(".quantiles"))
aux_parameters = set(p for n, p in model.named_parameters() if n.endswith(".quantiles"))
optimizer = optim.AdamW(parameters, lr=1e-4)
aux_optimizer = optim.AdamW(aux_parameters, lr=1e-3)
λ = 0.01

In [ ]:
epochs = 5
mb = master_bar(range(epochs))
mb.names = ['Distortion Loss', 'Rate Loss']
dist_losses, rate_losses = [], []
output_plot = Output()
display(output_plot)
model.train()

desired_batch_size = 24
actual_batch_size = 6
accumulation_steps = desired_batch_size // actual_batch_size

for i_epoch in mb:
    dataloader = DataLoader(train_dataset, batch_size=actual_batch_size, shuffle=True, drop_last=True, num_workers=6)
    
    for i, batch in enumerate(progress_bar(dataloader, parent=mb)):
        x = batch['audio'].permute(0,2,1).to("cuda").to(torch.float)        
        x_hat, y_likelihoods = model(x)
        N, _, L = x.size()
        num_samples = N * L
        rate_loss = torch.log(y_likelihoods).sum() / (-torch.log(torch.tensor(2)) * num_samples)
        dist_loss = F.mse_loss(x, x_hat)
        loss = dist_loss +  λ * rate_loss

        loss.backward()
        model.aux_loss().backward()
        if (i + 1) % accumulation_steps == 0: 
            optimizer.step()
            aux_optimizer.step()
            optimizer.zero_grad()
            aux_optimizer.zero_grad()

        dist_losses.append(np.log10(dist_loss.item()))
        rate_losses.append(np.log10(rate_loss.item()))
        
        if (i + 1) % accumulation_steps == 0:
            
            mb.update_graph(
                [[range(len(dist_losses)), dist_losses], [range(len(rate_losses)), rate_losses]],
                [0, i_epoch*len(dataloader)+i+1],
                [min(min(dist_losses), min(rate_losses)), max(max(dist_losses), max(rate_losses))]
            )
            with torch.no_grad():
                test_output = model.decode(model.encode(test_input).round())
                test_output = test_output.permute(0,2,1).cpu()

            with output_plot:
                output_plot.clear_output(wait=True)
                plt.figure(figsize=(16, 6),dpi=100)
                plt.plot(test_sample[0,39500:40000,0].numpy(), color='blue')
                plt.plot(test_sample[0,39500:40000,6].numpy(), color='red')
                plt.plot(test_output[0,39500:40000,0].numpy(), color='green',alpha=0.7)
                plt.plot(test_output[0,39500:40000,6].numpy(), color='purple',alpha=0.7)
                plt.show()